In [1]:
import pandas as pd 
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

1. webdriver를 오픈해서 'https://dart.fss.or.kr/' 요청을 보낸다. 
2. id가 'textCrpNm2' 태그를 선택 
3. 특정 입력값을 보낸다 (send_key()) -> ENTER 이벤트 발생
4. 해당 페이지는 table태그가 1개 존재 
    - 해당 페이지의 html 소스코드를 문자화
    - pandas 에 있는 read_html() 함수에 인자로 사용
    - read_html() 결과에서 첫번째 데이터프레임을 변수에 저장
5. 페이지 소스 코드를 BeautifulSoup으로 파싱
6. tbody 태그 중 id가 'tbody'인 태그를 선택 
7. tr 태그 들을 모두 검색한다. 
8. 각각 tr 태그에서 a태그를 모두 검색하고 두번째 a태그의 href 속성의 값을 추출
9. base_url과 href 속성의 값을 더해서 새로운 리스트에 추가 
10. read_html() 함수를 사용해서 나온 데이터프레임에 열 결합

In [2]:
# 기본 주소를 변수에 대입
base_url = 'https://dart.fss.or.kr/'

In [ ]:
# 웹 브라우저 오픈
driver = webdriver.Chrome()

In [4]:
# 웹 브라우저에 특정 주소를 입력
driver.get(base_url)

In [5]:
# id가 textCrpNm2 인 태그를 선택 
input_tag = driver.find_element( By.ID, 'textCrpNm2' )

In [6]:
# input_tag에 특정 데이터를 입력 
input_tag.send_keys('삼성전자')

In [7]:
# input_tag ENTER 이벤트 발생
input_tag.send_keys(Keys.ENTER)

In [8]:
# 해당 페이지의 페이지소스를 이용해서 데이터프레임 생성
# 페이지소스를 문자화
html_data = str(driver.page_source)
# 문자화된 페이지 소스를 read_html() 인자로 사용
# 결과의 첫번째 데이터프레임을 변수에 대입
df = pd.read_html(html_data)[0]

C:\Users\moons\AppData\Local\Temp\ipykernel_20672\3305765445.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_data)[0]


In [ ]:
df.head()

In [10]:
# 해당 페이지소스를 BeautifulSoup으로 파싱 
soup = bs(driver.page_source, 'html.parser')

In [12]:
# soup에서 tbody 태그 중 id가 tbody인 태그를 선택 
tbody_data = soup.find(
    'tbody', attrs={
        'id' : 'tbody'
    }
)

In [14]:
tr_list = tbody_data.find_all('tr')

In [19]:
# tr_list의 첫번째 데이터를 이용해서 반복 실행하는 구문을 테스트
base_url + tr_list[0].find_all('a')[1]['href']

'https://dart.fss.or.kr//dsaf001/main.do?rcpNo=20250604800428'

In [20]:
url_list = []
for tr_data in tr_list:
    url = base_url + tr_data.find_all('a')[1]['href']
    url_list.append(url)

In [ ]:
url_list

In [23]:
# df에 url_list 새로운 컬럼(url)에 대입 
df['url'] = url_list

In [ ]:
df.head()

In [25]:
# df를 csv로 저장 
df.to_csv('삼성전자.csv', index=False)

- 위의 코드들을 함수화
    - 첫번째 함수 
        - 매개변수 1개
            - driver : 웹 브라우져
        - table 태그를 찾아서 데이터프레임화
        - url 찾아서 데이터프레임에 대입 
        - 데이터프레임을 리턴

In [26]:
def create_df(web):
    # 해당 페이지의 페이지소스를 이용해서 데이터프레임 생성
    # 페이지소스를 문자화
    html_data = str(web.page_source)
    # 문자화된 페이지 소스를 read_html() 인자로 사용
    # 결과의 첫번째 데이터프레임을 변수에 대입
    df = pd.read_html(html_data)[0]
    # 해당 페이지소스를 BeautifulSoup으로 파싱 
    soup = bs(driver.page_source, 'html.parser')
    # soup에서 tbody 태그 중 id가 tbody인 태그를 선택 
    tbody_data = soup.find(
        'tbody', attrs={
            'id' : 'tbody'
        }
    )
    tr_list = tbody_data.find_all('tr')
    url_list = []
    for tr_data in tr_list:
        url = base_url + tr_data.find_all('a')[1]['href']
        url_list.append(url)
    # df에 url_list 새로운 컬럼(url)에 대입 
    df['url'] = url_list
    # 데이터프레임을 리턴
    return df
    

In [ ]:
create_df(driver)

- 두번째 함수 
    - 매개변수 1개 
        - 종목의 이름을 입력하는 공간 
    - 웹브라우져 오픈 
    - 다트에 접속
    - 종목을 입력하고 검색
    - create_df()함수 호출 
    - 함수의 결과를 저장 
    - 파일의 이름을 print() 출력

In [ ]:
def dart_save(_name):
    # 기본 주소를 변수에 대입
    base_url = 'https://dart.fss.or.kr/'
    driver = webdriver.Chrome()
    driver.get(base_url)
    input_tag = driver.find_element(By.ID , 'textCrpNm2')
    input_tag.send_keys(_name)
    input_tag.send_keys(Keys.ENTER)
    result = create_df(driver)
    result.to_csv(f"{_name}.csv", index=False)
    print(f"{_name}.csv 파일 생성 완료")
    driver.close()

In [29]:
dart_save('SK하이닉스')

C:\Users\moons\AppData\Local\Temp\ipykernel_20672\2203559859.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_data)[0]


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.104)
Stacktrace:
	GetHandleVerifier [0x0x7ff6b1effe75+79173]
	GetHandleVerifier [0x0x7ff6b1effed0+79264]
	(No symbol) [0x0x7ff6b1cb9e5a]
	(No symbol) [0x0x7ff6b1ca5c25]
	(No symbol) [0x0x7ff6b1ccac44]
	(No symbol) [0x0x7ff6b1d403c5]
	(No symbol) [0x0x7ff6b1d60922]
	(No symbol) [0x0x7ff6b1d38743]
	(No symbol) [0x0x7ff6b1d014c1]
	(No symbol) [0x0x7ff6b1d02253]
	GetHandleVerifier [0x0x7ff6b21ca2ad+3004797]
	GetHandleVerifier [0x0x7ff6b21c46fd+2981325]
	GetHandleVerifier [0x0x7ff6b21e3350+3107360]
	GetHandleVerifier [0x0x7ff6b1f1a9fe+188622]
	GetHandleVerifier [0x0x7ff6b1f2228f+219487]
	GetHandleVerifier [0x0x7ff6b1f08dc4+115860]
	GetHandleVerifier [0x0x7ff6b1f08f79+116297]
	GetHandleVerifier [0x0x7ff6b1eef528+11256]
	BaseThreadInitThunk [0x0x7ffd1ae4e8d7+23]
	RtlUserThreadStart [0x0x7ffd1c2bc34c+44]